<a href="https://colab.research.google.com/github/DavidB363/file-store/blob/master/Lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# My Recommender System

import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv')
print(df)

        74  -7.82   8.79  -9.66  -8.16  ...  99.22  -5.63  99.23  99.24  99.25
0      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
1       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
2       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
4      100  -6.17  -3.54   0.44  -8.50  ...  -3.35   0.05  -9.08  -5.05  -3.45
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24977  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24978   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24979   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24980   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24982 rows x 101 columns]


In [3]:
arr = df.values
print(arr.shape)

(24982, 101)


In [4]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1835282 (1835282,)


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

In [7]:
arr[idx[0][0], idx[1][0]]

6.21

In [8]:
new_arr[idx[0][0], idx[1][0]]

99.0